In [1]:
import torch
import gc
import time
from konlpy.tag import Okt

from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_classic.storage import InMemoryStore
from langchain_classic.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_classic.retrievers import EnsembleRetriever
from langchain_chroma import Chroma

In [2]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})
model = ChatOpenAI(model='gpt-5-mini', temperature=0)

In [3]:
loader = PyPDFLoader('./data/투자설명서.pdf')
docs = loader.load()
full_text = '\n\n'.join(doc.page_content for doc in docs)
text_splitter = SemanticChunker(embeddings=embeddings)
docs = text_splitter.create_documents([full_text])
for doc in docs:
    doc.metadata['source'] = '투자설명서.pdf'

In [4]:
okt = Okt()

def okt_tokenize(text):
    return okt.morphs(text, norm=True, stem=True)

In [5]:
bm25_retriever = BM25Retriever.from_documents(docs, preprocess_func=okt_tokenize, kwargs={'k':3})

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=bm25_retriever,
    chain_type='stuff',
    return_source_documents=True
)

In [7]:
qa_chain.invoke('이 회사가 발행한 주식의 총 발행량은 어느 정도야?')

{'query': '이 회사가 발행한 주식의 총 발행량은 어느 정도야?',
 'result': '공시상 현재 보통주식 발행주식수는 13,602,977주입니다.\n\n참고로 본 건 유상증자로 모집되는 신주는 7,500,000주(신주발행 예정)이므로, 유상증자까지 완료되면 총발행주식수는 최대 13,602,977 + 7,500,000 = 21,102,977주(예상)입니다.',
 'source_documents': [Document(metadata={'source': '투자설명서.pdf'}, page_content='주2) 2019년 9월 16일부터 전자증권제도가 시행됨에 따라 금번 유상증자 시 발행되는 신주인수권\n증서 및 신주가 전자증권으로 발행될 예정이며, 신주상장과 동시에 신주가 유통될 예정입니다. 주3) 상기 일정은 유관기관과의 협의 과정에 의하여 변경될 수 있습니다. 주4) 이베스트투자증권(주)는 2024년 6월 1일 이후 엘에스증권(주)로 사명이 변경되었습니다. 2. 공모방법 \n  \n[공모방법 : 주주배정후 실권주 일반공모] \n모집대상 주식수 비 고\n구주주\n(신주인수권증서 보\n유자)\n청약\n7,500,000주\n(100.0%)\n- 구주 1주당 신주 배정비율 : 1주당 0.55134989주\n- 신주배정 기준일 : 2024년 06월 28일\n- 구주주 청약일 : 2024년 08월 01일 ~ 2024년 08월 02일\n(2일간)\n- 보유한 신주인수권증서의 수량 한도로 청약가능(구주주에게\n는 신주배정기준일 현재 주주명부에 기재된 소유주식 1주당\n신주배정비율을 곱한 수량만큼의 신주인수권증서가 배정됨)\n- 신주인수권증서 거래를 통해서 신주인수권증서를 매매시 보\n유자 기준으로 청약 가능\n초과 청약 -\n- 「자본시장과 금융투자업에 관한 법률」 제165조의6의 제\n2항 제2호에 의거 초과청약\n- 초과청약비율 : 배정신주(신주인수권증서) 1주당 0.2주\n- 신주인수권증서 거래를 통해서 신주인수권증서를 매매시 보\n유자 기준으로 초과청약 가

In [8]:
persist_dir = './data/faiss_index_dense_2'

faiss_store = FAISS.from_documents(docs, embedding=embeddings)
faiss_store.save_local(persist_dir)

vectorstore = FAISS.load_local(persist_dir, embeddings=embeddings, allow_dangerous_deserialization=True)

In [9]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={'k':3})

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=faiss_retriever,
    chain_type='stuff',
    return_source_documents=True
)

In [11]:
qa_chain.invoke('이 회사가 발행한 주식의 총 발행량은 어느 정도야?')

{'query': '이 회사가 발행한 주식의 총 발행량은 어느 정도야?',
 'result': '발행주식총수는 13,602,977주입니다. (자기주식은 없어 유통주식수도 13,602,977주로 표기되어 있습니다.)',
 'source_documents': [Document(id='616475af-e511-45f1-935c-7830b8d98768', metadata={'source': '투자설명서.pdf'}, page_content='다. 다양한 종류의 주식 \n \n당사는 본 보고서 작성기준일 현재 해당 사항이 없습니다. 5. 정관에 관한 사항 \n \n가. 정관 변경 이력 \nⅠ. 발행할 주식의 총수 75,000,000 25,000,000 100,000,000 -\nⅡ. 현재까지 발행한 주식의 총수 13,602,977 - 13,602,977 -\nⅢ. 현재까지 감소한 주식의 총수 - - - -\n1. 감자 - - - -\n2. 이익소각 - - - -\n3. 상환주식의 상\n환 - - - -\n4. 기타 - - - -\nⅣ. 발행주식의 총수 (Ⅱ-Ⅲ) 13,602,977 - 13,602,977 -\nⅤ. 자기주식수 - - - -\nⅥ. 유통주식수 (Ⅳ-Ⅴ) 13,602,977 - 13,602,977 -\n정관변\n경일 해당주총명 주요변경사항 변경이유\n2020.03\n.25\n제14기 정기주\n주총회\n제2조(목적)\n제17조(전환사채의 발\n행)\n제18조(신주인수권부사\n채의 발행)\n제19조(이익참가부사채\n의 발행)\n제20조(교환사채의 발\n행)\n- 관련 사업 확장가능성에 따른 신규 목적사업\n추가\n- 한도 증액\n- 한도 증액\n- 한도 증액\n- 한도 증액\n2022.03\n.23\n제16기 정기주\n주총회\n제2조(목적)\n제8조의 2(주식 등의 전\n자등록)\n제47조(감사의 선임)\n- 관련 사업 확장가능성에 따른 신규 목적사업\n추가\n- 전자증권제도 도입 반영\n- 감사 선임에 관한 조문 정비 및 전자투표 도입\n시 결의 요건 내용 반